In [ ]:
batch
rn _G 효윻화


In [1]:
import pandas as pd
import tensorflow as tf
import itertools
import numpy as np
from tqdm import tqdm
# import helpers
import warnings
import pickle
import random
from tensorflow.contrib import slim
from tensorflow.contrib import rnn  #import LSTMCell, LSTMStateTupleTuple
warnings.filterwarnings('ignore')

In [2]:
with open('babi_preprocessed/val_dataset.pkl', 'rb') as f:
    train = pickle.load(f)

with open('babi_preprocessed/test_dataset.pkl', 'rb') as f:
    test = pickle.load(f)
    
# train = (Question, Answer, Context, Labels, Q_length, C_seq_length)

# max_ = 0
# for i in train[2]:
#     for j in i:
#         if len(j) > max_:
#             max_ = len(i)
# print(max_)

In [3]:
n_hidden = 32
n_classes = 159

q_vocab = 88
q_max_word = 12

c_vocal = 124
c_seq_num = 20
c_max_word = 12

batch_size = 16

question = tf.placeholder(tf.float32, [None, q_max_word ,q_vocab]) # [batch_size, num_word, one_hot dim] 
q_length= tf.placeholder(tf.int32, [None])

context = tf.placeholder(tf.float32, [None, c_seq_num, c_max_word, c_vocal]) 
                                                #  [batch_size, num_support_sentence, num_word, one_hot dim]
c_seq_length = tf.placeholder(tf.int32, [None,c_seq_num])
# c_num_seq = tf.placeholder(tf.int32, [None])

label = tf.placeholder(tf.int32, [None,c_seq_num, c_seq_num])

answer = tf.placeholder(tf.int32, [None, n_classes])

In [4]:
def question_LSTM(q, seqlen):
    q = tf.unstack(q, q_max_word,1)
    lstm_cell = rnn.LSTMCell(n_hidden)
    outputs, states = rnn.static_rnn(lstm_cell, q, dtype = tf.float32, sequence_length=seqlen)
    
    # change dimension to [batch_size, n_step, n_input]
    outputs = tf.stack(outputs)
    outputs = tf.transpose(outputs, [1,0,2])
    
#     # 색인생성을 위함, 나중에 label의 순서정보로 바꿔야하나?
#     # aymericdamien
    batchsize = tf.shape(outputs)[0]
    index = tf.range(0, batchsize) * q_max_word + (seqlen -1)
    outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)
    return outputs

In [5]:
def context_LSTM_helper(q, seqlen, reuse):
    with tf.variable_scope("cLSTM_helper", reuse=reuse) as scope:
        q = tf.unstack(q, c_max_word,1)
        lstm_cell = rnn.LSTMCell(n_hidden)
        outputs, states = rnn.static_rnn(lstm_cell, q, dtype = tf.float32, sequence_length=seqlen)

        # change dimension to [batch_size, n_step, n_input]
        outputs = tf.stack(outputs)
        outputs = tf.transpose(outputs, [1,0,2])

        batchsize = tf.shape(outputs)[0]
        index = tf.range(0, batchsize) * c_max_word + (seqlen -1)
        outputs = tf.gather(tf.reshape(outputs, [-1, n_hidden]), index)
        return outputs    

def context_LSTM(c, c_seqlen):
    result = []
    reuse = False
    for i in range(batch_size):
        sentences, sent_lengths = c[i], c_seqlen[i]
        tmp_out = context_LSTM_helper(sentences, sent_lengths,reuse)
        
        if not reuse:
            reuse = True
        
        result.append(tmp_out)
    return tf.stack(result)

In [6]:
def RN_g(object_i, object_j, question, reuse):
    # make object pair with question using tf.concat
    with tf.variable_scope('RN_g', reuse=reuse) as scope:
        opwq = tf.reshape(tf.concat([object_i, object_j, question], axis = 0), [1,-1])
        net = slim.fully_connected(opwq, 256 ) #reuse=reuse,  scope="g_1")
    #     net = slim.fully_connected(net, 256, reuse=reuse, scope="g_2")
    #     net = slim.fully_connected(net, 256, reuse=reuse, scope="g_3")
    #     net = slim.fully_connected(net, 256, reuse=reuse, scope="g_4")
        return net

In [7]:
def RN_f(element_sum, scope= "f_phi"):
    net = slim.fully_connected(element_sum, 159, scope="f_1")
#     net = slim.fully_connected(net, 512, scope="f_2")
#     net = slim.fully_connected(net, 159, scope="f_3")
    return net

In [8]:
def combination_nC2(objects):
    list_ = []
    for i in range(c_seq_num):
        list_.append(objects[i])
    return itertools.combinations(list_,2)

In [9]:
## garbage
def batch_load(data, batch_size):
    # data = (Question, Answer, Context, Labels, Q_length, C_seq_length)
    batch_q = random.sample(data[0], batch_size)
    batch_a = random.sample(data[1], batch_size)
    batch_c = random.sample(list(data[2]), batch_size)
    batch_l = random.sample(data[3], batch_size)
    batch_ql = random.sample(data[4], batch_size)
    batch_cl = random.sample(data[5], batch_size)
    return batch_q, batch_a, batch_c, batch_l, batch_ql, batch_cl

# label에 따른 순서 지정  
# context의 문장을 무조건 20개 사용(비효율적)->개선 할 수 있는 구현방법 

In [10]:
objects_units = context_LSTM(context, c_seq_length)
q_unit = question_LSTM(question, q_length)

In [11]:
objects_units

<tf.Tensor 'stack:0' shape=(16, 20, 32) dtype=float32>

In [12]:
q_unit

<tf.Tensor 'Gather:0' shape=(?, 32) dtype=float32>

In [11]:
# objects unit 을 run해서 그결과를 저장하고 combination을 만든 다음에  다시 feed dict 해서 그결과를 본다. 
result = []
reuse = False
for i in range(batch_size):
    first = True
    for (object_i,object_j) in combination_nC2(objects_units[i]):
        if first:
            element_summation = RN_g(object_i, object_j, q_unit[i], reuse = reuse)
            first = False
            reuse = True
        else :
            element_summation += RN_g(object_i,object_j,q_unit[i], reuse = reuse)
    result.append(element_summation)

result = tf.reshape(tf.stack(result), [batch_size, -1])
out = RN_f(result)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = out, labels = answer ))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)

correct_pred = tf.equal(tf.argmax(out,1), tf.argmax(answer,1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# training session

In [12]:
gpu_options = tf.GPUOptions(allow_growth=True, per_process_gpu_memory_fraction=0.1)
config = tf.ConfigProto(log_device_placement=True, allow_soft_placement=True,gpu_options=gpu_options)

with tf.Session(config= config) as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1,10000):
        batch_q, batch_a, batch_c, batch_l, batch_ql, batch_cl = batch_load(train, batch_size)

        if i % 100 == 0:
            train_accuracy = accuracy.eval(feed_dict = {question: batch_q, q_length: batch_ql, context: batch_c, c_seq_length: batch_cl, answer : batch_a})
            print('step %d, training accuracy %g' % (i, train_accuracy))
        optimizer.run(feed_dict = {question: batch_q, q_length: batch_ql, context: batch_c, c_seq_length: batch_cl, answer : batch_a})
    
    print("learning finished!")
    batch_q, batch_a, batch_c, batch_l, batch_ql, batch_cl = batch_load(test, batch_size)
    print('test accuracy %g' % accuracy.eval(feed_dict = {question: batch_q, q_length: batch_ql, context: batch_c, c_seq_length: batch_cl, answer : batch_a}))

step 100, training accuracy 0.125
step 200, training accuracy 0.1875
step 300, training accuracy 0.1875
step 400, training accuracy 0.0625
step 500, training accuracy 0.1875
step 600, training accuracy 0.125
step 700, training accuracy 0.125
step 800, training accuracy 0.125
step 900, training accuracy 0.0625
step 1000, training accuracy 0.0625
step 1100, training accuracy 0.0625
step 1200, training accuracy 0.1875
step 1300, training accuracy 0.1875
step 1400, training accuracy 0.0625
step 1500, training accuracy 0.125
step 1600, training accuracy 0.125
step 1700, training accuracy 0.1875
step 1800, training accuracy 0
step 1900, training accuracy 0
step 2000, training accuracy 0.25
step 2100, training accuracy 0.125
step 2200, training accuracy 0.0625
step 2300, training accuracy 0.125
step 2400, training accuracy 0.1875
step 2500, training accuracy 0.1875
step 2600, training accuracy 0.1875
step 2700, training accuracy 0.25
step 2800, training accuracy 0.125
step 2900, training accu